In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit.models import Model, GaussianModel

# Nulllage des Detektors

In [2]:
nulllage = pd.DataFrame([[180,180.5,181,181.5,181.75,182,182.25,182.5,183],
                         [2/60,2/60,2/60,2/60,2/60,2/60,2/60,2/60,2/60],
                         [11,110.4,500,1054,1179,1133,899,627,178.4],
                         [0.3,0.5,2,4,4,2,2,2,0.5]]).astype(float).T

In [3]:
nulllage

,0,1,2,3
0,180.00,0.033333,11.0,0.3
1,180.50,0.033333,110.4,0.5
2,181.00,0.033333,500.0,2.0
3,181.50,0.033333,1054.0,4.0
4,181.75,0.033333,1179.0,4.0
5,182.00,0.033333,1133.0,2.0
6,182.25,0.033333,899.0,2.0
7,182.50,0.033333,627.0,2.0
8,183.00,0.033333,178.4,0.5


In [4]:
mod_nullage = GaussianModel()

In [5]:
pars_nullage = mod_nullage.guess(nulllage[2],x=nulllage[0])
pars_nullage["center"].set(181)
pars_nullage["sigma"].set(1)

In [6]:
out_nullage = mod_nullage.fit(nulllage[2],params=pars_nullage,x=nulllage[0],weights=1/nulllage[3])

In [7]:
plt.plot(np.arange(179,184.2,0.1),out_nullage.eval(x=np.arange(179,184.2,0.1)),label="Fit")
plt.errorbar(nulllage[0],nulllage[2],xerr=nulllage[1],yerr=nulllage[3],fmt=' .',label="Messdaten")
plt.ylabel(r"Spannung / mV", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xlim([179.5,183.8])
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [8]:
print(out_nullage.fit_report())

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # function evals   = 32
    # data points      = 9
    # variables        = 3
    chi-square         = 314.304
    reduced chi-square = 52.384
[[Variables]]
    sigma:       0.60663646 +/- 0.002910 (0.48%) (init= 1)
    fwhm:        1.42851967 +/- 0.006852 (0.48%)  == '2.3548200*sigma'
    amplitude:   1802.21002 +/- 13.77597 (0.76%) (init= 1168)
    center:      181.815458 +/- 0.004419 (0.00%) (init= 181)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, center)         = -0.339 



In [9]:
# händisch bestimmtes maximum 
center = 181+42/60
spannung_center = 1212
err_spannung_center = 1
print(center)

181.7


# Winkelabhängigkeit der Intensität

In [10]:
Winkel = np.arange(0,400,45)
Intensität = np.array([1229,1230,1230,1231,1232,1226,1226,1232,1229])

In [11]:
Winkel

array([  0,  45,  90, 135, 180, 225, 270, 315, 360])

In [12]:
((Intensität.max()-Intensität.min())/Intensität.max())*100

0.48701298701298701

In [13]:
Intensität.max()-Intensität.min()

6

In [14]:
ax = plt.subplot(111, projection='polar')
ax.plot(np.deg2rad(Winkel), Intensität, ' h', color='r', linewidth=5, label="Messdaten")
ax.plot(np.arange(0,360,0.01),np.arange(0,360,0.01)/np.arange(0,360,0.01)*Intensität.mean(), color="g", alpha=0.6, linewidth=0.5, label="Mittelwert der Messdaten")
ax.set_rmax(1280)
ax.grid(True)
plt.legend(loc='lower center', bbox_to_anchor=(0.5, 0.18), fancybox=True, shadow=True)
plt.show()

# Fit

In [15]:
def get_err(a,b,a_d=2,b_d=2):
    return np.sqrt(np.square(a_d/b)+np.square((b_d*a)/b**2))

In [16]:
kupfer_messung = pd.read_csv("Ellipso/kupfer_2.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[:8]
kupfer_fit = pd.read_csv("Ellipso/kupfer_2.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[8:]
kupfer_p = np.array([1081,1013,952,886,878,916,883,917])
kupfer_s = np.array([1190,1203,1183,1101,1029,1162,970,946])
kupfer_messung.Weight = get_err(kupfer_p,kupfer_s)

In [71]:
kupfer_messung[kupfer_messung.Winkel.isin([60,80])]

,Winkel,Wert,Weight
1,80.0,0.843,0.002173
3,60.0,0.805,0.002332


In [17]:
alu_messung = pd.read_csv("Ellipso/alu.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[:7]
alu_fit = pd.read_csv("Ellipso/alu.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[7:]
alu_p = np.array([886,722,664,635,537,436,405])
alu_s = np.array([1083,984,883,791,599,431,401])
alu_messung.Weight = get_err(alu_p,alu_s)

In [58]:
alu_messung[alu_messung.Winkel.isin([60,80])]

,Winkel,Wert,Weight
1,80.0,0.735,0.002521
3,60.0,0.803,0.003242


In [18]:
ger_messung = pd.read_csv("Ellipso/germanium.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[:7]
ger_fit = pd.read_csv("Ellipso/germanium.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[7:]
ger_p = np.array([306,1039,125,226,289,327,340])
ger_s = np.array([1125,1062,911,755,610,415,346])
ger_messung.Weight = get_err(ger_p,ger_s)

In [59]:
ger_messung[ger_messung.Winkel.isin([60,80])]

,Winkel,Wert,Weight
1,80.0,0.099,0.002635
3,60.0,0.302,0.002765


In [19]:
glas_messung = pd.read_csv("Ellipso/glas.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[:6]
glas_fit = pd.read_csv("Ellipso/glas.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[6:]
glas_p = np.array([567,287,49,0.01,0.03,17.6])
glas_s = np.array([837,636,339,188,110,86.6])
glas_messung.Weight = get_err(glas_p,glas_s)

In [60]:
glas_messung[glas_messung.Winkel.isin([60,80])]

,Winkel,Wert,Weight
1,80.0,0.454,0.003450
3,60.0,0.017,0.010638


In [20]:
salz_messung = pd.read_csv("Ellipso/kochsalz.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[:7]
salz_fit = pd.read_csv("Ellipso/kochsalz.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[7:]
salz_p = np.array([423,163,9.4,2.7,2.3,1.2,0.001])
salz_s = np.array([644,387,81,63.7,45.3,90.1,70.2])
salz_messung.Weight = get_err(salz_p,salz_s)

In [61]:
salz_messung[salz_messung.Winkel.isin([60,80])]

,Winkel,Wert,Weight
1,80.0,0.427,0.005608
3,60.0,0.015,0.031425


In [62]:
si_messung = pd.read_csv("Ellipso/si.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[:7]
si_fit = pd.read_csv("Ellipso/si.DAT", delim_whitespace=True, names=["Winkel","Wert","Weight"])[7:]
si_p = np.array([258,35.7,25.7,104.5,150.5,175.3,152.1])
si_s = np.array([1093,1001,795,591,412,232,155.2])
si_messung.Weight = get_err(salz_p,salz_s)

In [64]:
si_messung[si_messung.Winkel.isin([60,80])]

,Winkel,Wert,Weight
1,80.0,0.039,0.005608
3,60.0,0.182,0.031425


In [29]:
plt.plot(kupfer_fit.Winkel,kupfer_fit.Wert, '-b', label="Fit")
plt.errorbar(kupfer_messung.Winkel, kupfer_messung.Wert,xerr=2/60,yerr=kupfer_messung.Weight, fmt=" .g", label="Messdaten")
plt.xlim([0,90])
plt.ylabel(r"R$_p$/R$_s$", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [ ]:
kupfer_n = 0.08-0.40
kupfer_k = 3.8-4.0

In [30]:
plt.plot(alu_fit.Winkel,alu_fit.Wert, '-b', label="Fit")
plt.errorbar(alu_messung.Winkel, alu_messung.Wert,xerr=2/60,yerr=alu_messung.Weight, fmt=" .g", label="Messdaten")
plt.xlim([0,90])
plt.ylabel(r"R$_p$/R$_s$", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [ ]:
alu_n = 1.36-1.70
alu_k = 7-8.1

In [36]:
plt.plot(ger_fit.Winkel,ger_fit.Wert, '-b', label="Fit")
plt.errorbar(ger_messung.Winkel, ger_messung.Wert,xerr=2/60,yerr=ger_messung.Weight*2, fmt=" .g", label="Messdaten")
plt.xlim([0,90])
plt.ylabel(r"R$_p$/R$_s$", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [ ]:
ger_n = 5.1-5.2
ger_k = 0.5-0.57

In [25]:
plt.plot(glas_fit.Winkel,glas_fit.Wert, '-b', label="Fit")
plt.errorbar(glas_messung.Winkel, glas_messung.Wert,xerr=2/60,yerr=glas_messung.Weight, fmt=" .g", label="Messdaten")
plt.xlim([0,90])
plt.ylabel(r"R$_p$/R$_s$", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [ ]:
glas_n = 1.456-1.550
glas_k = 0.00002

In [39]:
plt.plot(salz_fit.Winkel,salz_fit.Wert, '-b', label="Fit")
plt.errorbar(salz_messung.Winkel, salz_messung.Wert,xerr=2/60,yerr=salz_messung.Weight, fmt=" .g", label="Messdaten")
plt.xlim([0,90])
plt.ylabel(r"R$_p$/R$_s$", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [37]:
salz_n = 1.54

In [41]:
plt.plot(si_fit.Winkel,si_fit.Wert, '-b', label="Fit")
plt.errorbar(si_messung.Winkel, si_messung.Wert,xerr=2/60,yerr=si_messung.Weight/70, fmt=" .g", label="Messdaten")
plt.xlim([0,90])
plt.ylabel(r"R$_p$/R$_s$", fontsize=24)
plt.xlabel(r"Winkel / $^\circ$", fontsize=24)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best', fontsize=24)
plt.show()

In [40]:
si_n = 3.81-3.82
si_k = 0.013-0.017

In [75]:
1-2.2/3.8

0.42105263157894735

In [80]:
1-0.7/1.36

0.4852941176470589

In [82]:
1-0.6/1.36

0.5588235294117647

In [84]:
1-4.4/7

0.37142857142857133

In [85]:
1-4/7

0.4285714285714286

In [86]:
1-4/5.1

0.21568627450980382

In [87]:
1-3.7/5.1

0.27450980392156854

In [88]:
1-2/0.5

-3.0

In [89]:
1-2.2/0.5

-3.4000000000000004

In [90]:
1-1.4/1.456

0.03846153846153855

In [94]:
1-0.04/0.00002

-1998.9999999999998

In [95]:
1-1.51/1.54

0.019480519480519543

In [97]:
1-1.5/1.54

0.025974025974025983

In [ ]:
1-3.97/